The accelerometer data provided by NHANES has a measurement for each minute. This codes performs feature engineering to reduce data size and obtain meaningful physical activity features, and saves it in a new file (pax.csv)

If you want to skip this step, you can simply load the preprocessed data at pax.csv

In [1]:
import pandas as pd
from os import getcwd, listdir
from os.path import join
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from threading import Thread

## 1. Load NHANES 2013-2014 Physical Activity Monitor Data

In [2]:
#paxmin = pd.read_csv(join(getcwd(), "data", "csv", "Examination", "paxmin_h.csv"), chunksize=1000)
#test = paxmin.read()
paxmin = pd.read_sas(join(getcwd(), "data", "xpt", "Examination", "paxmin_h.xpt"))
paxday = pd.read_sas(join(getcwd(), "data", "xpt", "Examination", "paxday_h.xpt"))

Solving read_sas bug: #https://github.com/pandas-dev/pandas/issues/30051

In [ ]:
paxmin = paxmin.replace(5.397605346934028e-79,0) 

In [ ]:
paxday = paxday.replace(5.397605346934028e-79,0)

## 2. Extract features from day-by-day physical activity (PAXDAY)

In [ ]:
# Group the dataframe by SEQN (Subject ID)
grouped = paxday.groupby('SEQN')


# Define a custom weighted average function for each column of interest
def weighted_avg(data, values_col, weights_col):
    if data[weights_col].sum() == 0: # Some subjects have no valid minutes, set all features to NaN
        return np.nan
    return np.average(data[values_col], weights=data[weights_col])

# Apply the aggregations to the grouped object
day_features = grouped.apply(lambda x : pd.Series({
    'PAXDAYD': len(x), # Number of days with data (valid or not)
    'PAXVMD': weighted_avg(x, 'PAXVMD', 'PAXTMD'), # Valid Minutes per Day             (Daily average weighted by the number of total minutes per Day)
    'PAXWWMD': weighted_avg(x, 'PAXWWMD', 'PAXVMD'), # Wake Wear Minutes per Day       (Daily average weighted by the number of Valid Minutes per Day)
    'PAXSWMD': weighted_avg(x, 'PAXSWMD', 'PAXVMD'), # Sleep Wear Minutes per Day      (Daily average weighted by the number of Valid Minutes per Day)
    'PAXNWMD': weighted_avg(x, 'PAXNWMD', 'PAXVMD'), # Non-Wear Minutes per Day        (Daily average weighted by the number of Valid Minutes per Day)
    'PAXUMD': weighted_avg(x, 'PAXUMD', 'PAXVMD'), # Unknown activity Minutes per Day  (Daily average weighted by the number of Valid Minutes per Day)
    'PAXMTSD': weighted_avg(x, 'PAXMTSD', 'PAXWWMD'), # Day sum MIMS triaxial value    (Daily average weighted by the number of Wake Wear Minutes per Day)
    'PAXLXSD': weighted_avg(x, 'PAXLXSD', 'PAXWWMD') # Lux values (ambient light)      (Daily average weighted by the number of Wake Wear Minutes per Day)
})).reset_index()

## 3. Extract features from minute-by-minute physical activity (PAXMIN)

In [ ]:
paxmin['PAXQFM'] = paxmin['PAXQFM'].astype(int)

read_sas interprets some integers as byte literals, so we need to convert them back to integers

https://www.delftstack.com/howto/python/python-b-in-front-of-string/
https://www.geeksforgeeks.org/effect-of-b-character-in-front-of-a-string-literal-in-python/

In [ ]:
paxmin['PAXPREDM'] = paxmin['PAXPREDM'].apply(lambda x: int(x.decode('utf-8')))

In [ ]:
paxmin['PAXDAYM'] = paxmin['PAXDAYM'].apply(lambda x: int(x.decode('utf-8')))

Process only valid minutes

In [ ]:
valid = paxmin[(paxmin['PAXQFM'] == 0) & (paxmin['PAXPREDM'] == 1) & (paxmin['PAXMTSM'] >= 0)] #This filter extracts the valid minutes (rows). # Check if minute is valid (PAXQFM = 0) and wear time (PAXPREDM = 1), and PAXMTSM >= 0 (since PAXMTSM set as "-0.01" means that it could not be computed)

Let's aggregate minute-by-minute data for each day

In [ ]:
# Group the dataframe by SEQN (Subject ID) and PAXDAYM (Day of the week)
grouped = valid.groupby(['SEQN', 'PAXDAYM'])


# Define a custom function to compute the number of minutes at each intensity level
def counter(data, min_val, max_val):
    return ((data["PAXMTSM"] >= min_val) & (data["PAXMTSM"] < max_val)).sum()

# Define a custom function to compute the accumulated acceleration at each intensity level
def acceleration(data, min_val, max_val):
    return data[(data["PAXMTSM"] >= min_val) & (data["PAXMTSM"] < max_val)]["PAXMTSM"].sum() # PAXMTSM = MIMS triaxial value for the minute

# Thresholds for activity intensity (in MIMs)
Sedentary = 1
Light = 10
Moderate = 30
Vigorous = 45

# Apply the aggregations to the grouped object
auxiliary_features = grouped.apply(lambda x : pd.Series({
    'minutes': len(x),
    'time_Sedentary': counter(x, -np.inf, Sedentary),
    'time_Light': counter(x, Sedentary, Light),
    'time_Moderate': counter(x, Light, Moderate),
    'time_Vigorous': counter(x, Moderate, Vigorous),
    'time_Very Vigorous': counter(x, Vigorous, np.inf),
    'acc_Sedentary': acceleration(x, -np.inf, Sedentary),
    'acc_Light': acceleration(x, Sedentary, Light),
    'acc_Moderate': acceleration(x, Light, Moderate),
    'acc_Vigorous': acceleration(x, Moderate, Vigorous),
    'acc_Very Vigorous': acceleration(x, Vigorous, np.inf),
})).reset_index()

Now let's calculate the desired features (as weighted averages)

In [ ]:
grouped = auxiliary_features.groupby(['SEQN'])

# Define a custom weighted average function for each column of interest
def weighted_avg(data, values_col, weights_col):
    if data[weights_col].sum() == 0: # Some subjects have no valid wear minutes, set all features to NaN
        return np.nan
    return np.average(data[values_col], weights=data[weights_col])

minute_features = grouped.apply(lambda x : pd.Series({
        'Valid_days': len(x), # Number of Valid Days of Wear Time
        'PAXMINSB': weighted_avg(x, 'time_Sedentary', 'minutes'), # Minutes per Day of Sedentary Behaviour                      (Daily average weighted by the number of Wake Wear Minutes per Day)
        'PAXMINLPA': weighted_avg(x, 'time_Light', 'minutes'), # Minutes per Day of Light Physical Activity                     (Daily average weighted by the number of Wake Wear Minutes per Day)
        'PAXMINMPA': weighted_avg(x, 'time_Moderate', 'minutes'), # Minutes per Day of Moderate Physical Activity               (Daily average weighted by the number of Wake Wear Minutes per Day)
        'PAXMINVPA': weighted_avg(x, 'time_Vigorous', 'minutes'), # Minutes per Day of Vigorous Physical Activity               (Daily average weighted by the number of Wake Wear Minutes per Day)
        'PAXMINVVPA': weighted_avg(x, 'time_Very Vigorous', 'minutes'), # Minutes per Day of Very Vigorous Physical Activity    (Daily average weighted by the number of Wake Wear Minutes per Day)
        'PAXSUMSB': weighted_avg(x, 'acc_Sedentary', 'minutes'), # Sum of MIMs per Day at Sedentary Behaviour                   (Daily average weighted by the number of Wake Wear Minutes per Day)
        'PAXSUMLPA': weighted_avg(x, 'acc_Light', 'minutes'), # Sum of MIMs per Day at Light Physical Activity                  (Daily average weighted by the number of Wake Wear Minutes per Day)
        'PAXSUMMPA': weighted_avg(x, 'acc_Moderate', 'minutes'), # Sum of MIMs per Day at Moderate Physical Activity            (Daily average weighted by the number of Wake Wear Minutes per Day)
        'PAXSUMVPA': weighted_avg(x, 'acc_Vigorous', 'minutes'), # Sum of MIMs per Day at Vigorous Physical Activity            (Daily average weighted by the number of Wake Wear Minutes per Day)
        'PAXSUMVVPA': weighted_avg(x, 'acc_Very Vigorous', 'minutes'), # Sum of MIMs per Day at Very Vigorous Physical Activity (Daily average weighted by the number of Wake Wear Minutes per Day)
})).reset_index()

## 4. Store results in a dataframe

In [ ]:
# Merging day-by-day and minute-by-minute features
PAX_FEATURES = pd.merge(minute_features, day_features, on='SEQN', how='outer')
PAX_FEATURES = PAX_FEATURES.sort_values(by='SEQN')

In [ ]:
PAX_FEATURES.to_csv('pax.csv', index=False)

In the context of the given code, `minsb_days_vector` holds the number of sedentary minutes for each day, and `valid_wear_minutes` represents the total valid wear time (in minutes) for each day.

Now, let's imagine a real-life scenario to help illustrate the concept of the weighted average:

**Scenario**: Imagine three days of activity data.

1. **Day 1**: The subject was sedentary for 30 minutes, but the valid wear time was only 60 minutes.
2. **Day 2**: The subject was sedentary for 240 minutes (4 hours), but the valid wear time was 480 minutes (8 hours).
3. **Day 3**: The subject was sedentary for 60 minutes, and the valid wear time was 120 minutes (2 hours).

If we calculate a simple average of sedentary minutes, we'd get: \( \frac{30 + 240 + 60}{3} = 110 \) minutes, which means, on average, the person was sedentary for 110 minutes each day.

However, this doesn't account for the varying lengths of valid wear time each day. It treats a sedentary minute on a day where the device was only worn for 1 hour as equal to a sedentary minute on a day where it was worn for 8 hours. In reality, the day with 8 hours of wear time provides more information and should carry more weight.

By using a weighted average, we're saying: "Given the amount of time the device was validly worn each day, how many minutes was the person, on average, sedentary?"

So, in this example:
- Day 1 has a "weight" of 60 minutes.
- Day 2 has a "weight" of 480 minutes.
- Day 3 has a "weight" of 120 minutes.

The weighted average will prioritize Day 2's sedentary minutes more than Day 1's because the device was worn for a longer time on Day 2. 

**Intuitive Meaning**: The weighted average in this context essentially provides an average of sedentary minutes that is adjusted for the amount of time the device was validly worn each day. It gives us a more representative measure of sedentary behavior by accounting for the varying "confidence" or "reliability" of each day's data, as determined by wear time.